In [ ]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
import json
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("All libraries imported successfully")

# Cell 2: Load Data
df = pd.read_csv('../data/social_work_exam_dataset.csv')
print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head()

# Cell 3: Data Exploration
print("=" * 60)
print("DATA EXPLORATION")
print("=" * 60)

print("\nMissing Values:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("   No missing values!")
else:
    print(missing[missing > 0])

print(f"\nTarget Distribution:")
print(df['ExamResultPercent'].describe())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['ExamResultPercent'], bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Exam Result Percentage')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Exam Result Percentages')
axes[0].grid(alpha=0.3)

axes[1].boxplot(df['ExamResultPercent'])
axes[1].set_ylabel('Exam Score')
axes[1].set_title('Exam Score Boxplot')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Cell 4: Correlation Analysis
numerical_cols = ['Age', 'StudyHours', 'SleepHours', 'Confidence', 'TestAnxiety',
                 'MockExamScore', 'GPA', 'Scholarship', 'InternshipGrade', 'ReviewCenter', 'EnglishProficiency', 'MotivationScore', 'SocialSupport']

correlation_matrix = df[numerical_cols + ['ExamResultPercent']].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

target_corr = correlation_matrix['ExamResultPercent'].drop('ExamResultPercent').abs().sort_values(ascending=False)
print("\nTop Features Correlated with Exam Result Percentage:")
for feature, corr in target_corr.head(10).items():
    print(f"   {feature}: {corr:.4f}")

# Cell 5: Feature Importance Analysis
from sklearn.impute import SimpleImputer

print("=" * 60)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 60)

X = df.drop(['ExamResultPercent', 'Passed'], axis=1).copy()
y = df['ExamResultPercent'].values

categorical_cols = ['Gender', 'IncomeLevel', 'EmploymentStatus']
numerical_cols = ['Age', 'StudyHours', 'SleepHours', 'Confidence', 'TestAnxiety',
                 'MockExamScore', 'GPA', 'Scholarship', 'InternshipGrade', 'EnglishProficiency', 'MotivationScore', 'SocialSupport']
binary_cols = ['ReviewCenter']

print("\n[STEP 1] Handling Missing Values...")
print(f"Missing values before imputation:\n{X.isnull().sum()}")

imputer = SimpleImputer(strategy='median')
X[numerical_cols] = imputer.fit_transform(X[numerical_cols])

print(f"\nMissing values after imputation:\n{X.isnull().sum()}")
assert X.isnull().sum().sum() == 0, "ERROR: Still have missing values"
print("All missing values handled")

print("\n[STEP 2] Encoding Categorical Variables...")
le = LabelEncoder()
for col in categorical_cols:
    X[col] = le.fit_transform(X[col].astype(str))
    print(f"   Encoded: {col}")

assert not X.isnull().any().any(), "ERROR: NaN after encoding"
assert not np.isnan(y).any(), "ERROR: NaN in target"
print("Data encoding complete")

print("\n" + "=" * 60)
print("[METHOD 1] ANOVA F-test (f_regression)")
print("=" * 60)

selector_f = SelectKBest(score_func=f_regression, k='all')
selector_f.fit(X, y)

f_scores_df = pd.DataFrame({
    'Feature': X.columns,
    'F_Score': selector_f.scores_,
    'P_Value': selector_f.pvalues_
}).sort_values('F_Score', ascending=False)

print("\nANOVA F-test Results:")
for idx, row in f_scores_df.iterrows():
    sig = "***" if row['P_Value'] < 0.001 else "**" if row['P_Value'] < 0.01 else "*" if row['P_Value'] < 0.05 else ""
    print(f"   {row['Feature']:20s}: F={row['F_Score']:8.2f}, p={row['P_Value']:.4f} {sig}")

print("\n" + "=" * 60)
print("[METHOD 2] Mutual Information Regression")
print("=" * 60)

mi_scores = mutual_info_regression(X, y, random_state=42, n_neighbors=5)
mi_scores_df = pd.DataFrame({
    'Feature': X.columns,
    'MI_Score': mi_scores
}).sort_values('MI_Score', ascending=False)

print("\nMutual Information Scores:")
for idx, row in mi_scores_df.iterrows():
    print(f"   {row['Feature']:20s}: {row['MI_Score']:.6f}")

print("\n" + "=" * 60)
print("[METHOD 3] Pearson Correlation Analysis")
print("=" * 60)

pearson_results = []
for col in X.columns:
    corr_coef, p_value = pearsonr(X[col], y)
    pearson_results.append({
        'Feature': col,
        'Correlation': corr_coef,
        'Abs_Correlation': abs(corr_coef),
        'P_Value': p_value,
        'Significant': p_value < 0.05
    })

pearson_df = pd.DataFrame(pearson_results).sort_values('Abs_Correlation', ascending=False)

print("\nPearson Correlation Results:")
for idx, row in pearson_df.iterrows():
    sig = "***" if row['P_Value'] < 0.001 else "**" if row['P_Value'] < 0.01 else "*" if row['P_Value'] < 0.05 else ""
    print(f"   {row['Feature']:20s}: r={row['Correlation']:7.4f}, p={row['P_Value']:.4f} {sig}")

print("\n" + "=" * 60)
print("COMBINED FEATURE IMPORTANCE RANKING")
print("=" * 60)

combined = f_scores_df.merge(mi_scores_df, on='Feature')
combined = combined.merge(pearson_df[['Feature', 'Correlation', 'Abs_Correlation']], on='Feature')

combined['F_Rank'] = combined['F_Score'].rank(ascending=False, method='min')
combined['MI_Rank'] = combined['MI_Score'].rank(ascending=False, method='min')
combined['Corr_Rank'] = combined['Abs_Correlation'].rank(ascending=False, method='min')
combined['Avg_Rank'] = combined[['F_Rank', 'MI_Rank', 'Corr_Rank']].mean(axis=1)

combined = combined.sort_values('Avg_Rank')

print("\nTop 10 Most Important Features (by average rank):")
print(f"\n{'Rank':<6}{'Feature':<20}{'F-Score':<12}{'MI-Score':<12}{'Pearson r':<12}{'Avg Rank':<10}")
print("-" * 72)
for i, (idx, row) in enumerate(combined.head(12).iterrows(), 1):
    print(f"{i:<6}{row['Feature']:<20}{row['F_Score']:<12.2f}{row['MI_Score']:<12.6f}{row['Correlation']:<12.4f}{row['Avg_Rank']:<10.2f}")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].barh(f_scores_df['Feature'], f_scores_df['F_Score'], color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('F-Score', fontweight='bold')
axes[0, 0].set_title('ANOVA F-test Scores', fontweight='bold', fontsize=14)
axes[0, 0].invert_yaxis()
axes[0, 0].grid(axis='x', alpha=0.3)

axes[0, 1].barh(mi_scores_df['Feature'], mi_scores_df['MI_Score'], color='lightcoral', edgecolor='black')
axes[0, 1].set_xlabel('Mutual Information Score', fontweight='bold')
axes[0, 1].set_title('Mutual Information Regression', fontweight='bold', fontsize=14)
axes[0, 1].invert_yaxis()
axes[0, 1].grid(axis='x', alpha=0.3)

colors = ['green' if x > 0 else 'red' for x in pearson_df['Correlation']]
axes[1, 0].barh(pearson_df['Feature'], pearson_df['Correlation'], color=colors, edgecolor='black')
axes[1, 0].set_xlabel('Pearson Correlation', fontweight='bold')
axes[1, 0].set_title('Pearson Correlation with Target', fontweight='bold', fontsize=14)
axes[1, 0].invert_yaxis()
axes[1, 0].axvline(x=0, color='black', linestyle='--', linewidth=1)
axes[1, 0].grid(axis='x', alpha=0.3)

axes[1, 1].barh(combined.head(12)['Feature'], combined.head(12)['Avg_Rank'], color='mediumpurple', edgecolor='black')
axes[1, 1].set_xlabel('Average Rank (lower is better)', fontweight='bold')
axes[1, 1].set_title('Overall Feature Importance Ranking', fontweight='bold', fontsize=14)
axes[1, 1].invert_yaxis()
axes[1, 1].invert_xaxis()
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nFEATURE IMPORTANCE ANALYSIS COMPLETE")

# Cell 6: Data Preprocessing for Model Training
categorical_columns = ['Gender', 'IncomeLevel', 'EmploymentStatus']
numerical_columns = ['Age', 'StudyHours', 'SleepHours', 'Confidence', 'TestAnxiety',
                    'MockExamScore', 'GPA', 'Scholarship', 'InternshipGrade', 'EnglishProficiency', 'MotivationScore', 'SocialSupport']
binary_columns = ['ReviewCenter']

X_final = df[categorical_columns + numerical_columns + binary_columns].copy()
y_final = df['ExamResultPercent'].values

print("\n" + "=" * 60)
print("DATA PREPROCESSING FOR MODEL TRAINING")
print("=" * 60)

imputer_final = SimpleImputer(strategy='mean')
X_final[numerical_columns] = imputer_final.fit_transform(X_final[numerical_columns])

X_encoded = X_final.copy()
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    label_encoders[col] = le

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

assert not np.isnan(X_scaled).any(), "NaN after scaling"

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_final, test_size=0.2, random_state=42
)

print("\nData Preprocessing Complete")
print(f"   Training: {len(X_train)}, Test: {len(X_test)}")
print(f"   Features: {X_scaled.shape[1]}")
print(f"   Target range: [{y_train.min():.2f}, {y_train.max():.2f}]")

# Cell 7: Save Processed Data
import joblib

output_dir = '../preprocessed_dataset'
os.makedirs(output_dir, exist_ok=True)

print("\nFinal Verification:")
print(f"   X_train NaN count: {np.isnan(X_train).sum()}")
print(f"   X_test NaN count: {np.isnan(X_test).sum()}")

if np.isnan(X_train).any() or np.isnan(X_test).any():
    print("ERROR: Data contains NaN")
else:
    np.save(f'{output_dir}/X_train.npy', X_train)
    np.save(f'{output_dir}/X_test.npy', X_test)
    np.save(f'{output_dir}/y_train.npy', y_train)
    np.save(f'{output_dir}/y_test.npy', y_test)

    joblib.dump(scaler, f'{output_dir}/scaler.pkl')
    joblib.dump(label_encoders, f'{output_dir}/label_encoders.pkl')
    joblib.dump(imputer_final, f'{output_dir}/imputer.pkl')

    feature_names = categorical_columns + numerical_columns + binary_columns
    with open(f'{output_dir}/feature_names.json', 'w') as f:
        json.dump(feature_names, f)

    feature_importance_results = {
        'anova_f_test': f_scores_df.to_dict('records'),
        'mutual_information': mi_scores_df.to_dict('records'),
        'pearson_correlation': pearson_df.to_dict('records'),
        'combined_ranking': combined.to_dict('records')
    }

    with open(f'{output_dir}/feature_importance_analysis.json', 'w') as f:
        json.dump(feature_importance_results, f, indent=2)

    print(f"\nData saved to {output_dir}/")
    print("Feature importance results saved to feature_importance_analysis.json")
    print("PREPROCESSING COMPLETE")